# Using WAVE to secure a model

This notebook will set up the WAVE context for the integration demo. We assume that there are two parties: the one training the model, and the one using it.

First, we connect to wave:

In [1]:
import wave3 as wv
import grpc

# create the connection to wave
wave = wv.WAVEStub(grpc.insecure_channel("localhost:1410"))

Now we define a helper function that returns two WAVE entities that represents the granting organization and the receiving entity:

In [6]:
def createWaveEntities():
    orgEntity = wave.CreateEntity(wv.CreateEntityParams())
    _ = wave.PublishEntity(wv.PublishEntityParams(DER=orgEntity.PublicDER)) 
    
    recipientEntity = wave.CreateEntity(wv.CreateEntityParams())
    _ = wave.PublishEntity(wv.PublishEntityParams(DER=recipientEntity.PublicDER))
    
    return orgEntity, recipientEntity

We also define two methods that serialize and deserialize the WAVE entities, so we can be compatible with Flor.

In [1]:
import pickle

def serializeEntity(entity, file):
    ent_dict = {"pub": entity.PublicDER, "sec": entity.SecretDER, "hash": entity.hash}
    
    with open(file, 'wb') as f:
        f.write(pickle.dumps(ent_dict))

def deserializeEntity(file):
    with open(file, 'rb') as f:
        ent_dict = pickle.loads(f.read())
    
    return wv.CreateEntityResponse(PublicDER=ent_dict["pub"], SecretDER=ent_dict["sec"], hash=ent_dict["hash"])

Finally, we define a helper function that takes a granting entity, a recipient entity, and a resource name. The granting entity grants permission to the recipient entity to access the resource named.

In [9]:
def grantPermission(grantingEntity, recipientEntity, resource):
    _ = wave.CreateAttestation(wv.CreateAttestationParams(
        perspective=wv.Perspective(
            entitySecret=wv.EntitySecret(DER=grantingEntity.SecretDER)),
        subjectHash=recipientEntity.hash,
        publish=True,
        policy=wv.Policy(rTreePolicy=wv.RTreePolicy(
            namespace=grantingEntity.hash,
            indirections=5,
            statements=[
                wv.RTreePolicyStatement(
                    permissionSet=wv.WaveBuiltinPSET,
                    permissions=[wv.WaveBuiltinE2EE],
                    resource=resource,
                )]
        ))))